In [1]:
import os, csv
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
dataset = 'baby'
os.chdir('/home/*/XMMRec/' + dataset.title())
os.getcwd()

'/home/xin/XMMRec/Baby14'

In [15]:
# inter file preprocessed previously
inter = pd.read_csv(f'/home/**/data/{dataset}.inter', sep='\t')
print(f'shape: {inter.shape}')
inter[:4]

shape: (160792, 5)


,userID,itemID,rating,timestamp,x_label
0,0,1587,5.0,1373932800,0
1,0,1879,5.0,1373932800,0
2,0,0,5.0,1373932800,0
3,0,1922,5.0,1363996800,1


In [16]:
user=inter['userID'].unique()
user

array([    0,     1,     2, ..., 19442, 19443, 19444])

In [18]:
item=inter['itemID'].unique()
item

array([1587, 1879,    0, ..., 5958, 6845, 5855])

In [19]:
train=inter.loc[inter['x_label']==0]
val = inter.loc[inter['x_label']==1]
test = inter.loc[inter['x_label']==2]

In [20]:
train=train[['userID','itemID','rating']]
val=val[['userID','itemID','rating']]
test=test[['userID','itemID','rating']]

In [21]:
train[:10]

,userID,itemID,rating
0,0,1587,5.0
1,0,1879,5.0
2,0,0,5.0
5,1,2828,5.0
6,1,5403,5.0
7,1,1434,5.0
8,1,0,5.0
11,2,2846,2.0
12,2,6578,3.0
13,2,4011,5.0


In [22]:
train.rename(columns={'userID':'uid','itemID':'iid'}, inplace=True)

In [23]:
train['rating']=1

In [24]:
train

,uid,iid,rating
0,0,1587,1
1,0,1879,1
2,0,0,1
5,1,2828,1
6,1,5403,1
...,...,...,...
160783,19443,7032,1
160784,19443,7028,1
160787,19444,7022,1
160788,19444,6959,1


In [58]:
val.rename(columns={'userID':'uid','itemID':'iid'}, inplace=True)
test.rename(columns={'userID':'uid','itemID':'iid'}, inplace=True)
val['rating']=1
test['rating']=1

In [4]:
dir='DGVAE-baby/'

In [59]:


train.to_csv((dir+f'{dataset}.train.ratings'),index=False)
val.to_csv((dir+f'{dataset}.valid.ratings'),index=False)
test.to_csv((dir+f'{dataset}.test.ratings'),index=False)

In [5]:
from scipy.sparse import csr_matrix, load_npz

In [6]:
i_id, desc_str,title,brand, category = 'itemID', 'description','title','brand','categories'

file_name = f'meta-{dataset}.csv'

meta_file = os.path.join(file_name)

df = pd.read_csv(meta_file)
df.sort_values(by=[i_id], inplace=True)

print('data loaded!')
print(f'shape: {df.shape}')

df[:3]

data loaded!
shape: (7050, 10)


,itemID,asin,categories,description,title,price,imUrl,brand,related,salesRank
0,0,097293751X,[['Baby']],Easily keep track of your baby's or child's da...,"Baby Tracker&reg; - Daily Childcare Journal, S...",17.00,http://ecx.images-amazon.com/images/I/41Bb6wf%...,Time Too,"{'also_bought': ['9729375011', 'B004FN1AE8', '...",NaN
1,1,9729375011,[['Baby']],This is version of the award-winningBaby Track...,Newborn Baby Tracker&reg; - Round the Clock Ch...,15.95,http://ecx.images-amazon.com/images/I/51r3BLpL...,NaN,"{'also_bought': ['B000V5KPZ4', 'B001F8TLLU', '...",NaN
2,2,B00000IZQI,[['Baby']],This colorful car collection develops motor sk...,Fisher Price Nesting Action Vehicles,8.37,http://ecx.images-amazon.com/images/I/51E83QCC...,NaN,"{'also_bought': ['B0042D69W4', 'B00428LIZM', '...",NaN


In [7]:

# sentences: title + brand + category + description | All have title + description

title_na_df = df[df['title'].isnull()]
print(title_na_df.shape)

desc_na_df = df[df['description'].isnull()]
print(desc_na_df.shape)

na_df = df[df['title'].isnull() & df['description'].isnull()]
print(na_df.shape)

na3_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull()]
print(na3_df.shape)

na4_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull() & df['categories'].isnull()]
print(na4_df.shape)

(13, 10)
(570, 10)
(0, 10)
(0, 10)
(0, 10)


In [8]:

df['title'] = df['title'].fillna(" ")
df['description'] = df['description'].fillna(" ")
#df['ingredients'] = df['ingredients'].map(lambda x: x.replace('^',','))
df['brand'] = df['brand'].fillna(" ")
df['categories'] = df['categories'].fillna(" ")
df['description'][:10]

0    Easily keep track of your baby's or child's da...
1    This is version of the award-winningBaby Track...
2    This colorful car collection develops motor sk...
3    This darling cloth book offers hands-on experi...
4    In a relatively new concept in teething, The F...
5    As is the case with pacifiers and bottles, dif...
6    Wear your baby?! Dr. William Sears, noted pedi...
7    Original cotton blend fitted sheet specificall...
8    The Busy Bug Stroller bar is a portable stroll...
9    When dealing with a young baby or squirming to...
Name: description, dtype: object

In [9]:
# read img txt
topk_img2txt = 20
df_img = pd.read_csv(f'{dataset}-{topk_img2txt}.csv')
df_img['img2txt'] = df_img['img2txt'].fillna(" ")
df_img

,img2txt
0,babybook journal organizer layette diaper pid...
1,babybook breathablebaby babycare babyneat bab...
2,vtech playset playskool hape toys preschooler...
3,babybook pocket layette storybook babysoft pl...
4,teether teethers rattle teethifier teething p...
...,...
7045,vtech babyview soothers soother babyneat baby...
7046,breathablebaby cradle babycook babypote strol...
7047,bouncer babyview ergobaby ibaby babyzoo babys...
7048,carseat graco kidgear childproof gracotravel ...


In [10]:
img_texts = df_img['img2txt'].tolist()
img_texts

[' babybook journal organizer layette diaper piddlepad diapering nappy swaddlers diaperrx babyprints babycall swaddler babycare pottycover diapers organizers notes babyview babysoy',
 ' babybook breathablebaby babycare babyneat babyprints ergobaby ibaby babyhut babypote babyview mybaby layette babypro babysoy babyzoo babycall babygo babysling dreambaby babyganics',
 ' vtech playset playskool hape toys preschoolers babycare toy toyz carters toysmith childcare kinderwagon teethers infantino trikes toddlers kiddie tikes teether',
 ' babybook pocket layette storybook babysoft plushies babysoy book journal playskool cookbook pottycover babycook playtime cuteybaby plushie babysuite foldable blankie soother',
 ' teether teethers rattle teethifier teething pacifier teethe soother twister giggle pacifiers babypote breathablebaby toy grip soothers babycare kinderglo boppy babyzoo',
 ' teethers teether bead beads rattle chewbeads teethifier teethe pacifier teething bracelet pacifiers bracelets be

In [11]:
img_texts[0].split()[:5]

['babybook', 'journal', 'organizer', 'layette', 'diaper']

In [12]:
img_txt5 = []
for i in img_texts:
    tmp_s = ' '.join(i.split()[:5])
    img_txt5.append(tmp_s)
img_txt5

['babybook journal organizer layette diaper',
 'babybook breathablebaby babycare babyneat babyprints',
 'vtech playset playskool hape toys',
 'babybook pocket layette storybook babysoft',
 'teether teethers rattle teethifier teething',
 'teethers teether bead beads rattle',
 '',
 'cribwrap blankie babysoft swaddler mattress',
 'teether teethers rattle babyzoo twister',
 'humidifier sterilizer steriliser aspirator babypote',
 'cribwrap swaddler blankie foldable bassinet',
 'sleepwrap cribwrap sleepsack mattress babysoft',
 'pottycover tupperware humidifier steriliser sterilizer',
 'socket outlets plug adaptor adapter',
 'extender electrical adapter wireless adaptor',
 'soothers soother lids stoppers pacifiers',
 'latch latches locking fasteners clasp',
 'latches latch fasteners deadbolt locking',
 'fasteners safefit latch teethers latches',
 'teether soother teethifier strollerclips teethe',
 'pacifiers pacifier soother soothers teether',
 'playtub babypote pottypod babycook babysoft',


In [13]:
sentences = []
for i, row in df.iterrows():
    sen = row['title'] + ' ' + row['brand'] + ' '
    cates = eval(row['categories'])
    if isinstance(cates, list):
        for c in cates[0]:
            sen = sen + c + ' '
    sen += row[desc_str]
    sen += img_txt5[i]
    sen = sen.replace('\n', ' ')

    sentences.append(sen)

sentences[:10]

["Baby Tracker&reg; - Daily Childcare Journal, Schedule Log Time Too Baby Easily keep track of your baby's or child's daily schedules, activities and needs in one handy spot.  And make the drop-off and pick-up process for childcare a snap.  The Baby Tracker Daily Childcare Journal is designed to make it fast and easy to record and review meals, naps, activities, playtime, daily news, milestones and to-dos ---all in a single page view.  Get ready for doctor visits, give to childcare helpers and use this easy at-a-glance daily record to monitor schedules and prep for your baby's daily needs.  This version records daily activities from 6 a.m. to 7 p.m. and creates a visual snapshot of activities by time each day.   The Baby Tracker Daily Childcare Journal is ready to keep track of up to 6 months of daily schedule tracking (180 days) and is spiral bound to make it portable and lay flat for easy recording on a countertop or nightstand.Each journal includes:*180 daily at-a-glance tracking re

In [25]:
textual=[]
for u in user:
    sen=''
    #print(u)
    #print(list(inter[inter['userID']==u]['itemID']))
    ids=list(train[train['uid']==u]['iid'])
    for i in ids:
        sen+=sentences[i]
    #print(type(sen))
    #print(sen)
    textual.append(sen)
#print(textual)
print(len(textual))



19445


In [26]:

course_list = df[i_id].tolist()
#sentences = df[desc_str].tolist()

assert course_list[-1] == len(course_list) - 1

In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

tfidf_vec = TfidfVectorizer(min_df=5,max_df=0.6)

tfidf_matrix = tfidf_vec.fit_transform(textual)

print(tfidf_vec.get_feature_names())

print(tfidf_vec.vocabulary_)

['00', '000', '000000', '000203product', '000th', '001', '001bp9', '002', '004021', '0090', '00day', '00scd525', '00scd535', '01', '01a', '01abattery', '02', '02000', '02000z', '02040', '02040z', '0209', '02322', '02385', '03', '030', '039', '055121us', '055993', '05lbrollipop', '06aa10bks', '07', '0718', '076006us', '08', '082', '088', '09', '09770', '09771', '09772', '09775', '09776', '09777', '0c', '0m', '0mm', '0rethread', '0rganic', '10', '100', '1000', '1000ma', '1000th', '100pct', '100v', '101', '1027', '103', '104', '10401', '108', '109', '10inweight', '10mm', '10oz', '10pk', '10px', '10scd510', '10x', '11', '110', '110lbs', '110v', '111', '113', '114', '115', '117', '1171', '118', '11b', '11cm', '11in', '11px', '12', '120', '120range', '120v', '121', '1219', '123', '12351', '123s', '125', '125ml', '125px', '126', '128', '12cm', '12in', '12l', '12lbs', '12m', '12mb', '12pcs', '12pcswash', '12pm', '12px', '12v', '12vdc', '12x', '12x13', '12x16', '13', '130', '130ml', '13489', '1

/home/xin/.conda/envs/py37cuda113/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [28]:
import json
with open(dir+f'{dataset}.vocab', 'w') as outfile:
    json.dump(tfidf_vec.vocabulary_, outfile)

In [29]:
print(tfidf_matrix.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [30]:
text=csr_matrix(tfidf_matrix.toarray())

In [31]:
print(text)

  (0, 50)	0.0128292759104763
  (0, 70)	0.04922753035118759
  (0, 125)	0.024449197263927237
  (0, 184)	0.10079218198665386
  (0, 291)	0.02242629884007194
  (0, 447)	0.06491036652689663
  (0, 736)	0.02576985767393097
  (0, 790)	0.014976640085429012
  (0, 830)	0.02118275647025888
  (0, 908)	0.12638982378581118
  (0, 969)	0.04436546808926047
  (0, 1063)	0.020702469548906345
  (0, 1102)	0.03829730381616227
  (0, 1105)	0.03330334878186485
  (0, 1180)	0.01902924238491888
  (0, 1190)	0.023284821241407103
  (0, 1220)	0.02458056420175731
  (0, 1302)	0.02615868615104509
  (0, 1318)	0.013673869195847697
  (0, 1381)	0.06491036652689663
  (0, 1491)	0.04315272817606754
  (0, 1551)	0.04588682347096205
  (0, 1568)	0.020853952241709894
  (0, 1570)	0.025318433343083514
  (0, 1645)	0.017486079504556218
  :	:
  (19444, 13827)	0.11538560237371683
  (19444, 13831)	0.09069165859160476
  (19444, 13915)	0.10011105118251667
  (19444, 13984)	0.06611720624160539
  (19444, 14519)	0.12608728080235945
  (19444, 14626

In [32]:
print(text.shape)

(19445, 21340)


In [33]:
import scipy.sparse
scipy.sparse.save_npz(dir+f'{dataset}.user.words.tfidf.npz', text)

In [34]:
np.save(dir+f'{dataset}.uid',user)
np.save(dir+f'{dataset}.iid',item)
